# Rushing Offense/Defense Plus-Minus (ROPM and RDPM)
1. [Motivation](#motivation)
2. [Plus-Minus Formulation](#pm-formulation)
3. [Feature Engineering Player Exposures](#feature-engineering)
4. [Regression Results](#regression-results)
    * [ROPM Results](#ropm-results)
    * [RDPM Results](#rdpm-results)
5. [Discussion](#discussion)

## 1. Motivation <a class="anchor" id="motivation"></a>

Mainstream NFL analytics have a tendency of being rather unrevealing for a majority of players' on-field contributions.
Each play involves 22 players, yet only a handful will be assigned any credit or blame via classical counting statistics while the contributions of the remaining players are often left hopelessly latent from a data standpoint. 

It is our goal in the following analysis to construct a plus/minus (PM) statistic that quantifies the strengths and weaknesses of each and every participating NFL player, regardless of position.
Since the theme of this competition is tackling, we will restrict ourselves to rushing plays, the idea being that once a handoff occurs, the impending tackling of the ball carrier or its prevention becomes the primary focus of every on-field player.
**The two statistics presented in this analysis, Rushing Offense Plus-Minus (ROPM) and Rushing Defense Plus-Minus (RDPM), quantitatively grade every on-field player for their contributions, whether direct or indirect, to rushing-play tackles and are tailored for coaches' game-planning applications.**

## 2. Plus-Minus Formulation <a class="anchor" id="pm-formulation"></a>

Military strategists have formulated a framework for modeling and simulating the dynamics of battles through a series of "war equations".
This was first done via Lanchester's Laws and later generalized in Hughes' Salvo Combat Model.
The basic framework also provides a clean way to model and simulate sports contests, and it takes the following differential form:

$$
\small
\Delta S = \sum_i \alpha_i O_i - \sum_j \beta_j D_j + \epsilon \ \ . \ \ \ \ (1)
$$

Here, $\Delta S$ is the change in game score over some interval, $\vec{O}$ is exposure to the offensive side's players, $\vec{D}$ is exposure to the defending side's players, and $\epsilon$  is exposure to any random effect over the interval.
The coefficients $\vec{\alpha}$ and $\vec{\beta}$ are exactly the PM statistics we wish to define as they quantify the rate at which each player influences the game score per unit of on-field exposure.
Large positive values represent great effectiveness and large negative values represent great ineffectiveness.

PM statistics have become in vogue in NHL and NBA circles, yet remain comparatively absent from mainstream NFL analysis.
One reason for this absence involves the limited amount of mixing in NFL player rotations (e.g., quarterbacks and offensive linemen often play nearly every offensive snap in a game).
Another is that the context-critical nature of NFL plays can decouple basic counting statistics from game-score value (e.g., a 5 yard gain on 3rd & 10 in one's own territory has a much different value than a 5 yard gain on 3rd & 1 in the opponent's redzone).

We argue that the development of Expected Points Added (EPA) statistics and Player Tracking Data (PTD) enable the formulation of our ROPM and RDPM statistics via Eq. 1.
The context-inclusive nature of EPA makes it a natural choice for the target variable $\Delta S$, allowing us to appropriately value and incorporate any rushing play into our analysis.
Furthermore, PTD provides us with a powerful means for engineering player exposure feature variables $\vec{O}$ and $\vec{D}$ that are less collinear than simple on-field indicators.
The statistical estimation process is then equivalent to a single multivariate linear regression over historical data for all participating NFL players, with the resulting $\vec{\alpha}$ and $\vec{\beta}$ being our desired ROPM and RDPM statistics, respectively.

## 3. Feature Engineering Player Exposures <a class="anchor" id="feature-engineering"></a>

In basic PM statistics, the player exposure variables are often simple indicator functions for whether or not a player is on-field during an interval of play.
As previously mentioned, NFL player rotations don't lend themselves well to such definitions since many players typically play close to 100% of snaps alongside the same teammates.
Having this property in the fitting sample would lead to less powerful PM statistics since there would be few counterfactuals available to break the resulting collinearity among players, leading to ambiguity when quantifying players' relative contributions to a play result.

PTD provides us with the ability to define a more nuanced notion of player exposure that more directly quantifies each player's expected influence on a play result.
The definition we explore here is relatively simple and hopefully intuitive in an attempt to maximize the interpretability of the resulting ROPM and RDPM statistics.
It is roughly "the probability that a player influences a play" in the following sense:

$$
\small
O_i, D_i \equiv \sum_{x,y,t \ \in \ \text{play}} P(\text{player i influences (x,y) at t | ball carrier reaches (x,y) at t}) \cdot P(\text{ball carrier reaches (x,y) at t}) \ \ \ \ (2)
$$

for spatial coordinates $(x,y)$ and times $t$ since the snap of the ball, where $P(\cdot)$ represents a notion of probability.

Rigorously defining and calculating these probabilities is a difficult task, so we will instead approximate these terms via a heuristic approach.
The basic outline of our approach is to first infer the designed trajectory of a rushing play and to then integrate players' abilities to reach points along this trajectory based on their pre-snap alignments.
**We place emphasis on using only pre-snap information when calculating these exposures so that ROPM and RDPM have relevance for coaches when strategizing play-calling and player-alignment variations.**

We first find the basis that best describes the set of different classes of rushing plays via a Gaussian Mixture Model (GMM).
Specifically, we run the EM algorithm on the set of all designed rushing plays (i.e., ignoring QB scrambles) for weeks 1-9 of the 2022 NFL season.
The feature space for this inference is defined by the $(x,y)$ spatial coordinates for the first 1 second of post-handoff play, sampled at 0.1 second intervals.
We assume that there is minimal interaction between the ball carrier and other players during this interval so that the ball carrier’s trajectory closely follows its intended design.
Using the Bayesian Information Criterion to determine the optimal number of classes, we find that the following 7 trajectories best define the distinct classes of rushing plays:

![run_play_classes.png](https://rlynch91.github.io/nfl_big_data_bowl_2024/figures/run_play_classes.png)

The resulting GMM allows us to systematically label each rushing play in the sample set by the class that best describes it.
Using this labeled data, it is then straightforward to calculate the expected trajectory of each class at any post-handoff time, even beyond the first 1 second that we use to label the plays.
We can similarly calculate the percentage of rushing plays that survive to each time $t$ without a tackle having occurred.
The results can be used to proxy the probability of a ball carrier reaching each point along the designed play trajectory (i.e., the second term in the summation of Eq. 2), as depicted below.

![run_play_trajectories.png](https://rlynch91.github.io/nfl_big_data_bowl_2024/figures/run_play_trajectories.png)

We then proxy the players' abilities to influence a point $(x,y)$ on the designed play trajectory with their ability to reach it before the ball carrier’s expected arrival at time $t$ from the snap.
If they cannot reach the designed coordinate before the ball carrier, it is assumed they cannot influence that segment of the play nor meaningfully interact with any player who can.
These probabilities of influence (i.e., the first term in the summation of Eq. 2) are then determined solely by the distances between players' positioning at the time of the snap and the expected play trajectories.

Our estimators $\hat{O}$ and $\hat{D}$ of the offensive and defensive player exposures, respectively, then become

$$
\small
\hat{O}_i, \hat{D}_i = \frac{ \sum_t \mathbb{1}( (\text{distance between player i at snap and expected trajectory at t}) <= \lambda \cdot t ) \cdot P(\text{ball carrier reaching t})}{\sum_t P(\text{ball carrier reaching t})} \ \ \ \ (3)
$$

with $\lambda$ being an "effective max-speed" threshold that we select via cross-validation (greater detail on the tuning of this parameter is given in the Appendix).

**We emphasize that $\hat{O}$ and $\hat{D}$ do not take static values for each class of rushing play; they are calculated uniquely for all on-field players on every play based on their positions at the snap of the ball, as illustrated in the following two examples.**

![example_exposures.png](https://rlynch91.github.io/nfl_big_data_bowl_2024/figures/example_exposures.png)

**The variation of the player exposures based on player alignment and play design is what allows us to break collinearity among players, assigning more responsibility to players in closest proximity to the designed play trajectory.**
To give a sense of the typical scale of player exposure, the average values for each non-ball-carrier position group is given below for each class of rushing play.
As expected, interior linemen tend to have the highest exposure on inside rushing plays, while edge and skill players have more significant exposure on outside rushing plays.

![Ohat.png](https://rlynch91.github.io/nfl_big_data_bowl_2024/figures/Ohat.png)

![Dhat.png](https://rlynch91.github.io/nfl_big_data_bowl_2024/figures/Dhat.png)

## 4. Regression Results <a class="anchor" id="regression-results"></a>

We use all available data (i.e., weeks 1-9 of the 2022 NFL season) in our regression in order to maximize the quality of the resulting ROPM and RDPM statistics.
Details regarding the regression, its cross-validation, and its positive predictive power are covered in the Appendix; however, a few details are worth noting here.

First, we find that the cross-validated model performance is improved by pinning the player exposure $\hat{O}$ of quarterbacks and running backs to be a constant value of 1 when they are the ball carrier and 0 (i.e., uninvolved) otherwise.
For the purpose of this analysis, all other offensive players (e.g., wide receivers) are only ever treated as blockers.
These choices form 3 distinct responsibility groups on each rushing play:  1) ball-carriers (either QB or RB), 2) blockers (all other offensive players), and 3) tacklers (all defenders).

Second, because the regression is performed on over 1600 NFL players simultaneously, we include a Tikhonov (a.k.a., L2 or ridge) regularization term.
Doing so helps further break the collinearity among players, helps prevent overfitting, and naturally shrinks the contributions of low-snap players to a value of 0.

The results of the regression are illustrated below in two forms:  1) tables of the top-10 and bottom-10 ROPM and RDPM players at every position and 2) interactive charts of the player values for each team (see the Appendix for static renderings).
Only players that participated in at least 80 rushing plays are displayed, though values exist for every participating NFL player.
We scale ROPM and RDPM so that they represent the EPA players are expected to contribute on rushing plays over the course of an NFL game (approximately 25 rushing plays on average) if their exposures were fixed to 1.

> ### ROPM Results <a class="anchor" id="ropm-results"></a>

![ROPM_table.png](https://rlynch91.github.io/nfl_big_data_bowl_2024/figures/ROPM_table.png)

In [1]:
%%html
<iframe 
    src="https://rlynch91.github.io/nfl_big_data_bowl_2024/figures/ROPM_chart.html"
    width="1050" 
    height="525" 
    title="ROPM_chart" 
    style="border:none">
</iframe>

> ### RDPM Results <a class="anchor" id="rdpm-results"></a>

![RDPM_table.png](https://rlynch91.github.io/nfl_big_data_bowl_2024/figures/RDPM_table.png)

In [2]:
%%html
<iframe 
    src="https://rlynch91.github.io/nfl_big_data_bowl_2024/figures/RDPM_chart.html"
    width="1050" 
    height="525" 
    title="RDPM_chart" 
    style="border:none">
</iframe>

## 5. Discussion <a class="anchor" id="discussion"></a>

**ROPM and RDPM capture the effectiveness of post-snap interactions of players in terms of the tackling or the prevention of tackling of ball carriers on designed rushing plays.**
While the realized EPA value of a given rushing play is noisy, ROPM and RDPM capture the part of the distribution that is explainable by each player's pre-snap proximity to the designed play trajectory.
In this sense, both statistics represent tangible trends in the EPA data:  players will only rate out strongly if they are associated with positive-EPA plays.

ROPM and RDPM track the team rushing EPAs that they are constructed to explain.
For ball carriers, ROPM scales like EPA-per-carry and tracks more traditional counting statistics:  efficient runners like Lamar Jackson grade out quite highly while inefficient backs like Najee Harris grade out poorly.
For non-ball-carriers, players' impacts show up in the data as clear statistical trends even when their involvement in tackles is indirect.
For example, the highest-rated defender by RDPM is Daron Payne; the average Defensive EPA (i.e., negative Offensive EPA) is 0.26 on plays where his $\hat{D} > 0.5$, while it is -0.09 on plays where his $\hat{D} < 0.5$.
His value on rushing plays is significant even on plays where he is not the tackler, as shown in the animation below.
On a 3rd & 1 from the Washington 1 ([link](https://www.youtube.com/watch?v=FQO7QYJ9Jt0&t=6m50s)), Indianapolis chooses to run up the middle (M) directly into the strength of the Washington defense, a choice our model expects to lose Indianapolis 0.24 in predicted EPA.
By blocking Ryan Kelly (#78) into the backfield, Payne (#94) helps clear a lane for Jamin Davis (#52) to make a critical tackle-for-loss that costs Indianapolis 2.7 in realized EPA.

![Daron_Payne.gif](https://rlynch91.github.io/nfl_big_data_bowl_2024/figures/Daron_Payne.gif)

**Not only do ROPM and RDPM fortify the player coverage of NFL analytics, they are designed for coaches with game-planning applications in mind.**
I.e., the ROPM-RDPM regression model only requires pre-snap player alignments and designed play trajectory as inputs to predict EPA for a proposed play, and its positive predictive power generalizes to non-training data (see the Appendix for further discussion).
The per-team spectrum of ROPM-RDPM values suggests $O(1)$ gains in per-game EPA may be accessible by optimizating these inputs.
For example, in the above play, the model would only expect Indianapolis to lose 0.04 in predicted EPA if it were to instead run a play to the right sideline (R3) behind its highest-graded blocker Michael Pittman (#11); this is a 0.2 EPA improvement on a single play call.
ROPM and RDPM are not end-all statistics:  e.g., they are potentially vulnerable to biases introduced by scheme and player responsibility.
But their model could prove to be a useful complement to film study, identifying and quantifying statistical trends for further follow-up.

# Appendix
1. [Author](#author)
2. [Code](#code)
3. [Model Selection for Determining Run Classes](#model-selection)
4. [Regression Details](#regression-details)
5. [Static Results Charts](#static-charts)

## Author <a class="anchor" id="author"></a>

Ryan Lynch, PhD (MIT Physics, 2018)

rlynch@alum.mit.edu

## Code <a class="anchor" id="code"></a>

The code used for this analysis can be found in [this GitHub repo](https://github.com/rlynch91/nfl_big_data_bowl_2024/tree/main).  

## Model Selection for Determining Run Classes <a class="anchor" id="model-selection"></a>

When running the EM algorithm to find the maximum-likelihood GMM describing a dataset, one must specify the number of classes used in the mixture model.
Even though there is complete freedom in this choice, model selection can be performed to find the optimal number of classes according to some criteria.
In our analysis, model selection consists of minimizing the Bayesian Information Criterion (BIC), which penalizes models according to their number of free parameters (here, the number of classes in the GMM) in order to prevent overfitting.
In some sense, its minimization finds "the minimum number of classes needed to describe the data well."
As shown below, we find that the rushing trajectories included in our fitting sample are best described using 7 unique rushing classes in the GMM.

![EM_BIC.png](https://rlynch91.github.io/nfl_big_data_bowl_2024/figures/EM_BIC.png)

## Regression Details <a class="anchor" id="regression-details"></a>

The multivariate linear regression we perform to estimate ROPM and RDPM involves the minimization of a quadratic loss function, meaning the estimated regression coefficients are susceptible to being heavily influenced by outlier values if not properly handled.
In order to limit the extent to which outliers can dominate our estimation, we clip the values of EPA used in our sample to lie within the range (-3,3).
This approximately corresponds to clipping outlier values of EPA down to the 1% and 99% quantiles.

There are two free parameters in the regression model that need to be jointly tuned.
The relevant model selection is performed by maximizing the out-of-sample $R^2$ of a 3-fold cross-validation with a 67%-33% train-test split for each fold. 
This 3-fold cross-validation allows us to use all available data (i.e., weeks 1-9 of the 2022 NFL season) in our fits while still evaluating model performance on out-of-sample data.

The first free parameter is the $\lambda$ "max-speed" threshold mentioned above, which dictates the radius of a player's influence as a function of the time from snap. 
After experimenting with global versus position-specific values of $\lambda$, we find that a global value of 2.6 yards/second provides the best-fitting model. 
The second free parameter is a standard Tikhonov (a.k.a., L2 or ridge) regularization term, whose best-fit scale parameter is roughly 300.

The intercept parameter of the best-fit model takes a value of -0.03, reflecting the slightly negative expected value of EPA for the rushing plays included in the fitting sample.
Although ROPM and RDPM values exist for every participating NFL player, in this analysis we only share the results for players that participated in at least 80 rushing plays.
This threshold is chosen to select the primary participants for each team, and it corresponds to players that had a bye week during weeks 1-9 of the 2022 NFL season participating in an average of at least 10 rushing plays per game over the interval.
Due to their sparsity, we group players whose position is FB into the TE category when displaying results; however, we emphasize that this choice has no effect on the regression itself or the resulting ROPM and RDPM values. 

The $R^2$ of the best-fit model on the out-of-sample test data is 0.3%, which roughly corresponds to a positive correlation of 5.5% between the actual and predicted EPA on a per-play basis.
This value has the expected magnitude:  it is comparable to the predictive correlation of 4.8% needed to be profitable against the NFL spread in expectation ([proof given here](https://github.com/rlynch91/nfl_big_data_bowl_2024/blob/main/R2_Needed_to_Beat_NFL_Spread.ipynb)), another NFL-points prediction problem.
The $R^2$ value can likely be improved by scaling the regression variables by an EPA-volatility model that incorporates down, distance, and field position information; however, the leading-order effect of this change would likely be a global up/down-scaling of the ROPM and RDPM coefficients rather than a reordering of the coefficient ranks among players.
We can proxy a test of this assertion by varying our clipping range for EPA to in turn vary the influence of high-volatility regimes on the regression results, and we generally observe player ordering to be preserved when doing so.
Because player ranking and model interpretability are the primary goals of this analysis, we leave model improvements of a similar vein to future work.

## Static Results Charts <a class="anchor" id="static-charts"></a>

Here we provide the static renderings of the charts showing the spectrum of ROPM and RDPM player values for each team, with the top and bottom player on each team being named explicitly.

![ROPM_chart.png](https://rlynch91.github.io/nfl_big_data_bowl_2024/figures/ROPM_chart.png)

![RDPM_chart.png](https://rlynch91.github.io/nfl_big_data_bowl_2024/figures/RDPM_chart.png)